# Use Spark for Scala to load data and run SQL queries
This notebook introduces basic Spark concepts and helps you to start using Spark for Scala.

Some familiarity with Scala is recommended.

In this notebook, you'll use the publicly available **mtcars** data set from *Motor Trend* magazine to learn some basic Scala. You'll learn how to load data, create a Spark DataFrame, aggregate data, run mathematical formulas, and run SQL queries against the data.


## Table of contents
This notebook contains these main sections:

1. [Load a DataFrame](#Load_a_DataFrame)
2. [Initialize an SQLContext](#Initialize_an_SQLContext)
3. [Create a Spark DataFrame](#Create_a_Spark_DataFrame)
4. [Aggregate data after grouping by columns](#Aggregate_data_after_grouping_by_columns)
5. [Operate on columns](#Operate_on_columns)
6. [Run SQL queries from the Spark DataFrame](#Run_SQL_queries_from_the_Spark_DataFrame)

<a id='Load_a_DataFrame'></a>
## 1. Load a DataFrame
A DataFrame is a distributed collection of data that is organized into named columns. The local Scala DataFrame called **mtcars** includes observations on the following 11 variables:

`[, 1]	mpg     Miles / (US) gallon`  
`[, 2]	cyl     Number of cylinders`  
`[, 3]	disp	Displacement (cu. in.)`  
`[, 4]	hp      Gross horsepower`  
`[, 5]	drat    Rear axle ratio`  
`[, 6]	wt      Weight (1000 lbs)`  
`[, 7]	qsec    1/4 mile time (seconds)`  
`[, 8]	vs      0 = V-engine, 1 = straight engine`  
`[, 9]	am      Transmission (0 = automatic, 1 = manual)`  
`[,10]	gear    Number of forward gears`  
`[,11]	carb    Number of carburetors`

In [2]:
// Define the schema using a case class.
case class Cars(car: String, mpg: Double, cyl: Int, disp: Double, hp: Int, drat: Double, wt: Double, qsec: Double, 
                vs: Int, am: Int, gear: Int, carb: Int)

// Create some Cars data
val CarList: List[Cars] = List(
    Cars("Car Brand1", 18, 4, 1000, 110, 3.9, 3.1, 16.46, 1, 4, 4, 4),
    Cars("Car Brand2", 24, 6, 2000, 120, 3.8, 3.5, 16.46, 0, 55, 4, 4),
    Cars("Car Brand3", 30, 8, 3000, 130, 3.7, 4.0, 16.46, 1, 4, 4, 4),
    Cars("Car Brand4", 14, 4, 4000, 140, 3.6, 3.2, 16.46, 0, 55, 3, 4),
    Cars("Car Brand5", 12, 4, 5000, 150, 3.5, 3.3, 16.46, 1, 4, 4, 4),
    Cars("Car Brand6", 20, 6, 6000, 160, 3.4, 3.6, 16.46, 0, 55, 3, 4),
    Cars("Car Brand7", 15.5, 4, 700, 170, 3.3, 3.4, 16.46, 0, 1, 4, 4),
    Cars("Car Brand8", 10, 4, 800, 180, 3.2, 3.3, 16.46, 0, 0, 3, 4),
    Cars("Car Brand9", 5, 4, 900, 190, 3.1, 3.2, 16.46, 0, 1, 5, 4),
    Cars("Car Brand10", 7, 4, 2000, 110, 3.0, 3.1, 16.46, 0, 55, 3, 4),
    Cars("Car Brand11", 22, 6, 3000, 111, 2.9, 3.7, 16.46, 1, 4, 4, 4),
    Cars("Car Brand12", 33, 8, 4000, 112, 2.8, 4.2, 16.46, 0, 55, 4, 4),
    Cars("Car Brand13", 46, 8, 5000, 113, 2.7, 4.3, 16.46, 1, 4, 5, 4),
    Cars("Car Brand14", 8, 4, 6000, 114, 2.5, 3.2, 16.46, 0, 55, 4, 4),
    Cars("Car Brand15", 60, 8, 7000, 115, 2.4, 4.4, 16.46, 1, 4, 5, 4)
)

<a id='Initialize_an_SQLContext'></a>
## 2. Initialize an SQLContext
To work with a DataFrame, you need an SQLContext. You create this SQLContext by using `SQLContext(sc)`. A SparkContext named sc, which has been created for you, is used to initialize the SQLContext:

In [3]:
import au.com.bytecode.opencsv.CSVParser
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

<a id='Create_a_Spark_DataFrame'></a>
## 3. Create a Spark DataFrame
Using the SQLContext and the loaded local DataFrame, create a Spark DataFrame and print the schema, or structure, of the DataFrame:

In [4]:
val mtcars = sqlContext.createDataFrame(CarList)
mtcars.printSchema

root
 |-- car: string (nullable = true)
 |-- mpg: double (nullable = false)
 |-- cyl: integer (nullable = false)
 |-- disp: double (nullable = false)
 |-- hp: integer (nullable = false)
 |-- drat: double (nullable = false)
 |-- wt: double (nullable = false)
 |-- qsec: double (nullable = false)
 |-- vs: integer (nullable = false)
 |-- am: integer (nullable = false)
 |-- gear: integer (nullable = false)
 |-- carb: integer (nullable = false)



Display the content of the Spark DataFrame:

In [5]:
mtcars.show(32)

+-----------+----+---+------+---+----+---+-----+---+---+----+----+
|        car| mpg|cyl|  disp| hp|drat| wt| qsec| vs| am|gear|carb|
+-----------+----+---+------+---+----+---+-----+---+---+----+----+
| Car Brand1|18.0|  4|1000.0|110| 3.9|3.1|16.46|  1|  4|   4|   4|
| Car Brand2|24.0|  6|2000.0|120| 3.8|3.5|16.46|  0| 55|   4|   4|
| Car Brand3|30.0|  8|3000.0|130| 3.7|4.0|16.46|  1|  4|   4|   4|
| Car Brand4|14.0|  4|4000.0|140| 3.6|3.2|16.46|  0| 55|   3|   4|
| Car Brand5|12.0|  4|5000.0|150| 3.5|3.3|16.46|  1|  4|   4|   4|
| Car Brand6|20.0|  6|6000.0|160| 3.4|3.6|16.46|  0| 55|   3|   4|
| Car Brand7|15.5|  4| 700.0|170| 3.3|3.4|16.46|  0|  1|   4|   4|
| Car Brand8|10.0|  4| 800.0|180| 3.2|3.3|16.46|  0|  0|   3|   4|
| Car Brand9| 5.0|  4| 900.0|190| 3.1|3.2|16.46|  0|  1|   5|   4|
|Car Brand10| 7.0|  4|2000.0|110| 3.0|3.1|16.46|  0| 55|   3|   4|
|Car Brand11|22.0|  6|3000.0|111| 2.9|3.7|16.46|  1|  4|   4|   4|
|Car Brand12|33.0|  8|4000.0|112| 2.8|4.2|16.46|  0| 55|   4| 

Try different ways of retrieving subsets of the data. For example, get the first 5 values in the **mpg** column:

In [6]:
mtcars.select("mpg").show(5)

+----+
| mpg|
+----+
|18.0|
|24.0|
|30.0|
|14.0|
|12.0|
+----+
only showing top 5 rows



Filter the DataFrame to retain only rows with **mpg** values that are less than 18:

In [7]:
mtcars.filter(mtcars("mpg") < 18).show()

+-----------+----+---+------+---+----+---+-----+---+---+----+----+
|        car| mpg|cyl|  disp| hp|drat| wt| qsec| vs| am|gear|carb|
+-----------+----+---+------+---+----+---+-----+---+---+----+----+
| Car Brand4|14.0|  4|4000.0|140| 3.6|3.2|16.46|  0| 55|   3|   4|
| Car Brand5|12.0|  4|5000.0|150| 3.5|3.3|16.46|  1|  4|   4|   4|
| Car Brand7|15.5|  4| 700.0|170| 3.3|3.4|16.46|  0|  1|   4|   4|
| Car Brand8|10.0|  4| 800.0|180| 3.2|3.3|16.46|  0|  0|   3|   4|
| Car Brand9| 5.0|  4| 900.0|190| 3.1|3.2|16.46|  0|  1|   5|   4|
|Car Brand10| 7.0|  4|2000.0|110| 3.0|3.1|16.46|  0| 55|   3|   4|
|Car Brand14| 8.0|  4|6000.0|114| 2.5|3.2|16.46|  0| 55|   4|   4|
+-----------+----+---+------+---+----+---+-----+---+---+----+----+



<a id='Aggregate_data_after_grouping_by_columns'></a>
## 4. Aggregate data after grouping by columns
Spark DataFrames support a number of common functions to aggregate data after grouping. For example, you can compute the average weight of cars as a function of the number of cylinders:

In [8]:
import org.apache.spark.sql.functions._
mtcars.groupBy("cyl").agg(avg("wt")).show()

+---+-------+
|cyl|avg(wt)|
+---+-------+
|  6|    3.6|
|  4|  3.225|
|  8|  4.225|
+---+-------+



<p>You can also sort the output from the aggregation to determine the most popular cylinder configuration in the DataFrame:</p>

In [9]:
mtcars.groupBy("cyl").agg(count("wt")).sort($"count(wt)".desc).show()

+---+---------+
|cyl|count(wt)|
+---+---------+
|  4|        8|
|  8|        4|
|  6|        3|
+---+---------+



<a id='Operate_on_columns'></a>
## 5. Operate on columns
Scala provides a number of functions that you can apply directly to columns for data processing. In the following example, a basic arithmetic function converts lbs to metric tons:

In [20]:
mtcars.withColumn("wtTon", mtcars("wt") * 0.45).select(("car"),("wt"),("wtTon")).show(6)

+----------+---+------------------+
|       car| wt|             wtTon|
+----------+---+------------------+
|Car Brand1|3.1|             1.395|
|Car Brand2|3.5|             1.575|
|Car Brand3|4.0|               1.8|
|Car Brand4|3.2|1.4400000000000002|
|Car Brand5|3.3|1.4849999999999999|
|Car Brand6|3.6|              1.62|
+----------+---+------------------+
only showing top 6 rows



<a id='Run_SQL_queries_from_the_Spark_DataFrame'></a>
## 6. Run SQL queries from the Spark DataFrame
You can register a Spark DataFrame as a temporary table and then run SQL queries over the data. The `sql` function enables an application to run SQL queries programmatically and returns the result as a DataFrame:

In [11]:
mtcars.registerTempTable("cars")

val highgearcars = sqlContext.sql("SELECT car, gear FROM cars WHERE gear <= 5")
highgearcars.show()

+-----------+----+
|        car|gear|
+-----------+----+
| Car Brand1|   4|
| Car Brand2|   4|
| Car Brand3|   4|
| Car Brand4|   3|
| Car Brand5|   4|
| Car Brand6|   3|
| Car Brand7|   4|
| Car Brand8|   3|
| Car Brand9|   5|
|Car Brand10|   3|
|Car Brand11|   4|
|Car Brand12|   4|
|Car Brand13|   5|
|Car Brand14|   4|
|Car Brand15|   5|
+-----------+----+



In [12]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f


Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


Name: Compile Error
Message: <console>:1: error: unclosed character literal
%%brunel data('mtcars') x("car") y("grear") bar
                     ^
StackTrace: 

In [46]:
val cyl_dis = sqlContext.sql("select avg(mpg) as avg_mpg, avg(wt) as avg_wt, cyl, car from cars group by car, cyl")
cyl_dis.show(5)

+-------+------+---+-----------+
|avg_mpg|avg_wt|cyl|        car|
+-------+------+---+-----------+
|   33.0|   4.2|  8|Car Brand12|
|   18.0|   3.1|  4| Car Brand1|
|    7.0|   3.1|  4|Car Brand10|
|   12.0|   3.3|  4| Car Brand5|
|   46.0|   4.3|  8|Car Brand13|
+-------+------+---+-----------+
only showing top 5 rows



In [50]:

%%brunel data('cyl_dis')  bar x(cyl) y(avg_wt) sort(avg_wt) + line x(cyl) y(avg_wt)

In [59]:
%%brunel data('cyl_dis') cloud x(car) size(avg_mpg) color(avg_wt) sort(avg_mpg) 

In [58]:
%%brunel data('cyl_dis')  chord x(car) y(car) size(avg_mpg) color(avg_wt)

In [63]:
%%brunel data('cyl_dis') bubble label(car) size(avg_mpg) color(car)

## That's it!
You successfully completed this notebook! You learned how to load a DataFrame, view and filter the data, aggregate the data, perform operations on the data in specific columns, and run SQL queries against the data. For more information about Spark, see the [Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html).

<div class="alert alert-block alert-info"> Note: To save resources and get the best performance please use the code below to stop the kernel before exiting your notebook.</div>

In [11]:
%%javascript
Jupyter.notebook.session.delete();

<hr>
Copyright &copy; IBM Corp. 2017. Released as licensed Sample Materials.